In [1]:
%pylab inline
import numpy as np
from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time
import sys
import os
import torch
import torch.nn as nn
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))

from Rmse_loss import rmse_loss 
from All import all0
from Select import select
from Train_fun import train_fun

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
def diff_burgers(net,cor_train):
    cache = np.zeros((1,6))
    pts = cor_train.clone().detach().to(device)
    pts.requires_grad_(True)
    n_iter = len(pts)//10000
    for i in range(n_iter+1):
        pts0 = pts[10000*i:10000*(i+1)]
        outs = net(pts0)
        grad = torch.autograd.grad(outs, pts0, grad_outputs=torch.ones_like(outs), create_graph=True)[0]
        ut_t = (grad[:,0]).reshape(-1,1)
        ux_t = (grad[:,1]).reshape(-1,1)
        uxx_t  = ((torch.autograd.grad(ux_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxx_t  = ((torch.autograd.grad(uxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxxx_t  = ((torch.autograd.grad(uxxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)

        u_pred = outs.cpu().detach().numpy()
        ut = ut_t.cpu().detach().numpy()
        ux = ux_t.cpu().detach().numpy()
        uxx = uxx_t.cpu().detach().numpy()
        uxxx = uxxx_t.cpu().detach().numpy()
        uxxxx = uxxxx_t.cpu().detach().numpy()
        del ut_t, ux_t, uxx_t, uxxx_t, uxxxx_t, outs
        torch.cuda.empty_cache()
        cache = np.vstack((cache,np.hstack((u_pred,ux,uxx,uxxx,uxxxx,ut))))
    return cache[1:]

In [3]:
class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64,64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [4]:
def parametric_burgers_rhs(u, t, params):
    k,a,b,c = params
    deriv = a*(1+c*np.sin(np.pi/4*t))*u*ifft(1j*k*fft(u)) + b*ifft(-k**2*fft(u))
    return real(deriv)

In [5]:
# Set size of grid
n = 500
m = 201

# Set up grid
x = np.linspace(-8,8,n+1)[:-1];   dx = x[1]-x[0]
t = np.linspace(0,10,m);          dt = t[1]-t[0]
k = 2*np.pi*fftfreq(n, d = dx)

# Initial condition
u0 = np.exp(-(x+1)**2)

# Solve with time dependent uu_x term
params = (k, -1, 0.1, 0.5)
u = odeint(parametric_burgers_rhs, u0, t, args=(params,)).T

u_xx_true = 0.1*np.ones(m)
uu_x_true = -1*(1+0.5*np.sin(np.pi/4*t))
X, T = np.meshgrid(x, t)

In [6]:
data = np.hstack((T.reshape(-1,1), X.reshape(-1,1), u.T.reshape(-1,1))).astype(float32)
data = data[data[:,0]//0.05%3==1]
data = data[data[:,1]//0.032%5==1]

In [16]:
t = data[:,0].reshape(-1,1)
x = data[:,1].reshape(-1,1)
u_clean = data[:,-1].reshape(-1,1)
per = 0.2
np.random.seed(0)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')

In [17]:
x_train = torch.from_numpy(data[:,:-1])
u_train = torch.from_numpy(u_noise).reshape(-1,1)
net = FiveNet_sin(2).to(device)
try:
    net.load_state_dict(torch.load('net/net_{}'.format(per)))
except:
    train_fun(net,x_train,u_train,N_red_lr=4,epochs=3000,lr=0.001,threshold=0.001)
    torch.save(net.state_dict(),'net/net_{}'.format(per))

In [18]:
cache = diff_burgers(net,x_train)
u_pred = cache[:,0].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uxx = cache[:,2].reshape(-1,1)
uxxx = cache[:,3].reshape(-1,1)
uxxxx = cache[:,4].reshape(-1,1)
ut = cache[:,-1].reshape(-1,1)

In [19]:
uu_x_true = -1*(1+0.5*np.sin(np.pi*t/4))

In [20]:
rmse_loss(uu_x_true*u_pred*ux+0.1*uxx,ut)

tensor(0.1772, device='cuda:0', dtype=torch.float64)

In [21]:
feature = np.hstack((t,x,u_pred,ux,uxx,uxxx,ut))
LC,exp_list,exp = all0('result_{}.txt'.format(per),feature,
                              state=3,deg=2,alpha=10**-5,tol=0.05,
                              data_points=1000,niterations=1000)

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 5.870e+05
Head worker occupation: 8.0%
Progress: 1310 / 15000 total iterations (8.733%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           4.273e-03  1.594e+01  y = -0.0032677
3           1.119e-03  6.699e-01  y = (-0.41046 * x₃)
5           7.859e-04  1.767e-01  y = ((-0.057573 - x₃) * x₂)
7           4.027e-04  3.343e-01  y = ((0.081963 * x₄) - (x₃ * x₂))
9           3.286e-04  1.017e-01  y = ((0.081963 * x₄) - (x₃ * (x₂ + 0.065313)))
11          3.282e-04  6.734e-04  y = (((0.081963 * 1.0242) * x₄) - (x₃ * (x₂ + 0.065313)))
12          3.207e-04  2.296e-02  y = sin((x₄ * 0.094628) - ((x₃ * x₂) * 1.1915))
14          3.094e-04  1.794e-02  y = ((((x₄ - cube(x₃)) * 0.080338) - (x₃ * x₂)) / 0.88032)
16          3.092e-04  3.629e-04  y = ((((x₄ - cube(x₃ * 0.93572)) * 0.080338) - (x₃ * x₂)) / 0....
                               

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.765e-01, tolerance: 3.713e-03
  model = cd_fast.enet_coordinate_descent(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.225e-03, tolerance: 3.713e-03
  model = cd_fast.enet_coordinate_descent(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

1 tensor(0.2120, device='cuda:0', dtype=torch.float64)
2 tensor(0.2120, device='cuda:0', dtype=torch.float64)


/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.840e-01, tolerance: 3.713e-03
  model = cd_fast.enet_coordinate_descent(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.225e-03, tolerance: 3.713e-03
  model = cd_fast.enet_coordinate_descent(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

...

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.939e-01, tolerance: 3.713e-03
  model = cd_fast.enet_coordinate_descent(
/home/chr/SI-SR/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


[[0.         1.         0.         0.         0.         0.        ]
 [0.         2.         0.03249322 0.03249322 0.00012292 0.00012292]
 [0.         3.         0.01631946 0.01631946 0.0001187  0.0001187 ]
 [0.         4.         0.16745567 0.16745567 0.00176889 0.00176889]
 [0.         5.         0.         0.         0.         0.        ]
 [1.         2.         0.05251916 0.05251916 0.00012292 0.00012292]
 [1.         3.         0.02587571 0.02587571 0.0001187  0.0001187 ]
 [1.         4.         0.26117117 0.26117117 0.00176889 0.00176889]
 [1.         5.         0.         0.         0.         0.        ]
 [2.         3.         0.01552064 0.01556114 0.00001746 0.00026024]
 [2.         4.         0.02671202 0.02278304 0.00275215 0.00223165]
 [2.         5.         0.01179024 0.01179024 0.00012292 0.00012292]
 [3.         4.         0.02513418 0.02324369 0.00274293 0.00222863]
 [3.         5.         0.00595139 0.00595139 0.0001187  0.0001187 ]
 [4.         5.         0.06577892

3 tensor(0.2564, device='cuda:0', dtype=torch.float64)
4 tensor(0.2564, device='cuda:0', dtype=torch.float64)
[ 0.          0.          0.          0.          0.          0.
 -1.20108952  0.10384965  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
[[0.         1.         0.         0.         0.         0.        ]
 [0.         2.         1.76739607 1.76739607 0.00011939 0.00011939]
 [0.         3.         0.16467978 0.16467978 0.00170287 0.00170287]
 [0.         4.         0.         0.         0.         0.        ]
 [1.         2.         2.83331021 2.83331021 0.00011939 0.00011939]
 [1.         3.         0.25684177 0.25684177 0.00170287 0.00170287]
 [1.         4.         0.         0.         0.         0.        ]
 [2.         3.         0.23042259 0.19374762 0.0027227  0.00219041]
 [2.         4.         0.61093131 0.61093131 0.00011939 0.00011939]


/home/chr/SI-SR/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 5.480e+05
Head worker occupation: 7.6%
Progress: 1232 / 15000 total iterations (8.213%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           4.725e-03  1.594e+01  y = -0.0025115
3           3.029e-03  2.223e-01  y = (0.050577 * x₃)
5           3.023e-03  9.903e-04  y = ((0.050569 * x₃) + -0.0024508)
7           2.577e-03  7.979e-02  y = (((-0.026502 * x₁) - -0.083657) * x₃)
9           2.573e-03  7.190e-04  y = (((-0.026871 * x₁) - -0.090056) * (x₃ + -0.026871))
10          2.495e-03  3.112e-02  y = (0.20401 / ((exp(x₁) + 1.1221) / x₃))
12          2.454e-03  8.294e-03  y = (0.20401 / (((exp(x₁) - -1.2583) / x₃) - 0.047494))
14          2.298e-03  3.278e-02  y = (((-0.026871 * (x₁ - (-0.026871 / exp(x₃)))) - -0.090056) ...
                                  * x₃)
16          2.216e-03  1.803e-02  y = (((-0.026871 * (x₁ - (x₃ / (ex

In [22]:
k = select(loss_comp,0.05)
exp_list[k]

[1 1 1 1 1 1 1 1 1 1 1]
[ 1  1  1  1  1  1  1  1  1 -1]
[ 1 -1 -1  1 -1 -1  1  1 -1]
[-1 -1  1 -1 -1  1  1 -1]
[-1  1  1  1  1  1  1]
[1 1 1 1 1 1]
[-1 -1  1  1  1]
[1 1 1 1]
[1 1 1]
[ 1 -1]
[-1]


-0.434771928053452*x2*sin(0.796965*x0) - x2 + 0.106388890676161*x3